In [ ]:
#For Kaggle
#date=''

#For WL data
date='20241118'
source='WL'

saveImg=False

print('done')

In [ ]:
import sys
import os
from importlib import reload
fpath = os.path.join('..//scripts')
sys.path.append(fpath)

import warnings
warnings.filterwarnings('ignore')

#loading internal scripts
import frauddetection as fd
import sourcedata as sd
import dataimport as di
import countrymanagement as cm
import mccmanagement
reload(fd)
reload(sd)
reload(di)
reload(cm)
reload(mccmanagement)

print('done')

In [ ]:
max_silouhette=0
silouhette_config=''
print('max silouhette and context',max_silouhette,silouhette_config )

In [ ]:
import pandas as pd


PATH="../data/raw/"
merchants = pd.read_csv(PATH+'WLacceptorIdSummary30d.csv', sep=";")
print(merchants.shape)
merchants.head()

In [ ]:
merchants.isnull().sum()


In [ ]:
merchants['country_group'] = merchants['term_country'].apply(cm.get_country_group)
#merchants= merchants.drop(columns=['term_country'])
merchants.head()

In [ ]:
merchants=pd.get_dummies(merchants,columns=['country_group'], dtype = int)

In [ ]:
import numpy as np
import seaborn as sns

# Creating plot
sns.boxplot(data=merchants,y='count_amount')

In [ ]:
import matplotlib.pyplot as plt

# An "interface" to matplotlib.axes.Axes.hist() method
plt.hist(x=merchants['count_amount'], bins=75, color='#0504aa', alpha=0.7, rwidth=0.85)
plt.grid(axis='y', alpha=0.75)
plt.xlabel('Value (sum amount')
plt.ylabel('Frequency')
plt.title('Card holder sum amount histogram')

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

merchants['count_amount_log10']=np.log10(1+merchants['count_amount'])
merchants['sum_amount_log10']=np.log10(1+merchants['sum_amount'])

# Creating plot
sns.boxplot(data=merchants,y='count_amount_log10')
plt.title('merchant sum amount log10 box plot')

plt.show()

In [ ]:
import matplotlib.pyplot as plt

# An "interface" to matplotlib.axes.Axes.hist() method
plt.hist(x=merchants['count_amount_log10'], bins=75, color='#0504aa', alpha=0.7, rwidth=0.85)
plt.grid(axis='y', alpha=0.75)
plt.xlabel('Value (sum amount')
plt.ylabel('Frequency')
plt.title('Merchant sum amount histogram')

plt.show()

In [ ]:
import matplotlib.pyplot as plt

# An "interface" to matplotlib.axes.Axes.hist() method
plt.hist(x=merchants['ecom_rate'], bins=75, color='#0504aa', alpha=0.7, rwidth=0.85)
plt.grid(axis='y', alpha=0.75)
plt.xlabel('Value (ecom rate')
plt.ylabel('Frequency')
plt.title('Merchant sum amount histogram')

plt.show()

In [ ]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

scalers = fd.getScalers()
merchants2=merchants.drop(['acceptor_id', 'sum_amount', 'count_amount','term_mcc','term_country'], axis=1)
print(merchants2.head(5))
for key in scalers:
    print(key)
    scaler=scalers.get(key)
    merchants3 = scaler.fit_transform(merchants2)
    #merchants4 = pd.DataFrame(merchants3)
   
    kmeans = KMeans(n_clusters=3, random_state=42)
    kmeans.fit(merchants3)
    y_pred = kmeans.predict(merchants3)
    pred = pd.DataFrame(y_pred)
    print(pred[0].value_counts())
    print('silhouette score',silhouette_score(merchants3,y_pred, sample_size=5000))
    print('-----------------')

In [ ]:
max_silouhette=0

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

range_n_clusters = np.arange(2,5,1)
print(range_n_clusters)


scalers =StandardScaler()
merchants2=merchants.drop(['acceptor_id', 'sum_amount', 'count_amount','term_mcc','term_country'], axis=1)
merchants3 = scaler.fit_transform(merchants2)


#print(card3.head())
ssd=[]
silhouettes=[]
for n_clusters in range_n_clusters: 
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans.fit(merchants3)
    y_pred = kmeans.predict(merchants3)
    
    #cards3["label"]=y_pred
    pred = pd.DataFrame(y_pred)
    print(pred[0].value_counts())
    ssd.append(kmeans.inertia_)    
    score = silhouette_score(merchants3,y_pred, sample_size=5000)
    silhouettes.append(score)
    if(n_clusters>3 and score>max_silouhette):
        max_silouhette=score
        silouhette_config=''+str(n_clusters)+' StandardScaler'
    print('--', n_clusters, score)
print('-----------------')
print('max silouhette and context',max_silouhette,silouhette_config )

In [ ]:
fig, ax = plt.subplots() # Create the figure and axes object
# Plot the first x and y axes:
ax.plot(range_n_clusters, ssd) 
ax.set_ylabel("ssd",color="b")
ax2 = ax.twinx()

ax2.plot(range_n_clusters, silhouettes, color='r') 
ax2.set_ylabel("silhouette",color="r")

In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import silhouette_score

range_n_clusters = np.arange(2,5,1)
print(range_n_clusters)


scalers =MinMaxScaler()
merchants2=merchants.drop(['acceptor_id', 'sum_amount', 'count_amount','term_mcc','term_country'], axis=1)
merchants3 = scaler.fit_transform(merchants2)


#print(card3.head())
ssd=[]
silhouettes=[]
for n_clusters in range_n_clusters: 
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans.fit(merchants3)
    y_pred = kmeans.predict(merchants3)
    
    #cards3["label"]=y_pred
    pred = pd.DataFrame(y_pred)
    print(pred[0].value_counts())
    ssd.append(kmeans.inertia_)    
    score = silhouette_score(merchants3,y_pred, sample_size=5000)
    silhouettes.append(score)
    if(n_clusters>3 and score>max_silouhette):
        max_silouhette=score
        silouhette_config=''+str(n_clusters)+' MinMaxScaler'
    print('--', n_clusters, score)
print('-----------------')
print('max silouhette and context',max_silouhette,silouhette_config )

In [ ]:
fig, ax = plt.subplots() # Create the figure and axes object
# Plot the first x and y axes:
ax.plot(range_n_clusters, ssd) 
ax.set_ylabel("ssd",color="b")
ax2 = ax.twinx()

ax2.plot(range_n_clusters, silhouettes, color='r') 
ax2.set_ylabel("silhouette",color="r")

In [ ]:
import numpy as np
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import silhouette_score

range_n_clusters = np.arange(2,5,1)
print(range_n_clusters)


scalers =MinMaxScaler()
merchants2=merchants.drop(['acceptor_id', 'sum_amount', 'count_amount','term_mcc','term_country'], axis=1)
merchants3 = scaler.fit_transform(merchants2)


#print(card3.head())
ssd=[]
silhouettes=[]
for n_clusters in range_n_clusters: 
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans.fit(merchants3)
    y_pred = kmeans.predict(merchants3)
    
    #cards3["label"]=y_pred
    pred = pd.DataFrame(y_pred)
    print(pred[0].value_counts())
    ssd.append(kmeans.inertia_)    
    score = silhouette_score(merchants3,y_pred, sample_size=5000)
    silhouettes.append(score)
    if(n_clusters>3 and score>max_silouhette):
        max_silouhette=score
        silouhette_config=''+str(n_clusters)+' RobustScaler'
    print('--', n_clusters, score)
print('-----------------')
print('max silouhette and context',max_silouhette,silouhette_config )

In [ ]:
fig, ax = plt.subplots() # Create the figure and axes object
# Plot the first x and y axes:
ax.plot(range_n_clusters, ssd) 
ax.set_ylabel("ssd",color="b")
ax2 = ax.twinx()

ax2.plot(range_n_clusters, silhouettes, color='r') 
ax2.set_ylabel("silhouette",color="r")

# Final choice StandardScaler nb cluster =4

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score


scalers =StandardScaler()
 

merchants2=merchants.drop(['acceptor_id', 'sum_amount', 'count_amount','term_mcc','term_country'], axis=1)
merchants3 = scaler.fit_transform(merchants2)
merchants3=pd.DataFrame(merchants3)
#print(merchants3.head())
merchants3.columns=['ecom_rate','recency','country_group_BELGIUM','country_group_EUROPE','country_group_WORLD','count_amount_log10','sum_amount_log10']


kmeans = KMeans(n_clusters=4, random_state=42)
kmeans.fit(merchants3)
y_pred = kmeans.predict(merchants3)
    
#cards3["label"]=y_pred
pred = pd.DataFrame(y_pred)
print(pred[0].value_counts())
  
score = silhouette_score(merchants3,y_pred, sample_size=5000)
silhouettes.append(score)
print('--', n_clusters, score)
print('-----------------')
result = merchants.join(pred)
result[0]='M'+result[0].map(str)
result.head(5)


In [ ]:
import numpy as np

# Creating plot
sns.boxplot(data=result,y='count_amount_log10', hue=0)

In [ ]:
import numpy as np

# Creating plot
sns.boxplot(data=result,y='recency', hue=0)

In [ ]:
import numpy as np

# Creating plot
sns.boxplot(data=result,y='sum_amount_log10', hue=0)

In [ ]:
result2=result.drop(['sum_amount','count_amount','ecom_rate','recency','country_group_BELGIUM','country_group_EUROPE','country_group_WORLD','count_amount_log10','sum_amount_log10'] , axis=1)
result2.columns=['acceptor_id','term_mcc','term_country', 'clusterMerchant']
result2['acceptor_id']=result2['acceptor_id'][:-1]
result2.head()

In [ ]:
#result2.to_csv('../data/processed/WLAcceptorProfile.csv', index=False)